In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *
from setting import *

In [ ]:
w = pd.read_table(PATH_DICT["w.tsv"], index_col=0)

h = pd.read_table(PATH_DICT["h.tsv"], index_col=0)

In [ ]:
for w_or_h, df in (("w", w.T), ("h", h)):

    k_hcc = ccal.hierarchical_consensus_cluster_with_multiple_k(
        df,
        HCC_KS,
        n_job=MAX_N_JOB,
        distance_function=ccal.compute_correlation_distance,
        plot_df=df.shape[1] < PLOT_MAX_SIZE ** 0.5,
        directory_path=PATH_DICT["{}|hcc/".format(w_or_h)],
    )

    ks = tuple("K{}".format(k) for k in HCC_KS)

    hcc__k_x_column = pd.read_table(
        PATH_DICT["{}|hcc__k_x_column.tsv".format(w_or_h)], index_col=0
    )

    for k, element_cluster in hcc__k_x_column.iterrows():

        if w_or_h == "w":

            xaxis_title = FEATURE_ALIAS

        elif w_or_h == "h":

            xaxis_title = SAMPLE_ALIAS